In [1]:
import matplotlib.pyplot as plt
import matplotlib

%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2


import numpy as np
#%matplotlib inline

import os
from utils import *

import os

from utils_calcium import ProcessCalcium, get_reward_triggered_psth, plot_multi_session_psth_imshow, plot_psth

Autosaving every 180 seconds


/home/cat/.conda/envs/bmitools/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [8]:
####################################################
################## LOAD DATA #######################
####################################################

root_dir = '/media/cat/8TB/donato/bmi/'

animal_ids = [
    # M1 mice
    "DON-011733",    # M1    - Processed
    #"DON-014618",    # M1    - Processed
    #'DON-014451',    # M1    - Processed
    #'DON-014382',    # M1    - Processed
    #'DON-014575',    # M1    - Processed
    #'DON-014618',    # M1    - Processed
    #'DON-015821',    # M1    - Processed
    # 'DON-017923',   # M1    - Not Processed <has extra days at the end....
    # 'DON-018129',   # M1    - Not Processed
     #'DON-018130',   # M1    - Not Processed

    # MAYBE MICE
    # 'DON-013392'  <-- only last day was good, all others were at chance
    # 'DON-015962'  <-- might be ok, rechedk it.

    # CA3 mice
    #"DON-012502",    # CA3    - spreadsheet is old - need reprocessing; ALSO Weird performance, may exclude
    #"DON-014266",    # CA3   - Processed
    #'DON-014371',    # CA3   - Processed
    #'DON-015801',    # CA3   - Processed
    #'DON-016658',    # CA3   - Processed
    #"DON-019683",    # CA3   - Processed
]
#

animal_id = animal_ids[0]

#
c = ProcessCalcium(root_dir,
                    animal_id)

# # 
c.remove_bad_cells = False
c.use_non_merged = True

#
c.recompute_binarization = False  # IF TRue: must input correct params from bin notebooks
c.dff_min = 0.05                  # min %DFF for [ca] burst to considered a spike (default 5%) overwrites percentile threshold parameter
c.percentile_threshold = 0.99   # this is pretty fixed, we don't change it; we want [ca] bursts that are well outside the "physics-caused"noise
c.maximum_std_of_signal = 0.25     # if std of signal is greater than this, then we have a noisy signal and we don't want to binarize it

#
c.save_figures = c.recompute_binarization
c.load_sessions()

#
print ("....DONE...")


  0%|          | 0/8 [00:00<?, ?it/s]

 ***** USING non-merged binarization
remove bad cells:  True
...setting 2p parameters...


 12%|█▎        | 1/8 [00:01<00:11,  1.69s/it]


 ***** USING non-merged binarization
remove bad cells:  True
...setting 2p parameters...


 25%|██▌       | 2/8 [00:03<00:10,  1.71s/it]

...


 ***** USING non-merged binarization
remove bad cells:  True
...setting 2p parameters...


 88%|████████▊ | 7/8 [00:14<00:02,  2.24s/it]


 ***** USING non-merged binarization
remove bad cells:  True
...setting 2p parameters...


100%|██████████| 8/8 [00:16<00:00,  2.07s/it]


....DONE...


In [12]:
import networkx as nx
import pickle
import yaml


#
def compute_graph_metrics(animal_id, 
                          recompute_graphs=False, 
                          pcorr_thresh=0.99, 
                          use_median=False):
    
    # load yaml file and get session_ids field
    fname_yaml = os.path.join(root_dir, 
                              animal_id, 
                              str(animal_id)+'.yaml')
    
    # load yaml file
    d = yaml.load(open(fname_yaml), 
                  Loader=yaml.FullLoader)
    
    # get session ids
    session_ids = d['session_ids']



    # load all the pairwise distributions
    diameters = []
    clustering = []
    aspl = []
    degree = []
    pagerank = []
    ctr = 0
    for session_id in session_ids[1:]:
        
        #
        dir_main = os.path.join(root_dir, 
                                animal_id, 
                                str(session_id),
                                'plane0',
                                #'merged',
                                'correlations')
        
        #
        fname_out = os.path.join(dir_main, 'graph_metrics.npz')
       #print ("fname_out: ", fname_out)

        if os.path.exists(fname_out)==False or recompute_graphs:

            # load all the .npz files from dir_main
            files = os.listdir(dir_main)
            # delte all files that have "graph" in filename
            files = [file for file in files if 'graph' not in file]
            #print ("# files: ", len(files))

            # initialize connectivity matrix
            cm = np.zeros((len(files), len(files)), 'float32')

            #
            for k in range(len(files)):

                # load cell correlations
                fname_pcorr = os.path.join(dir_main, files[k])
                d = np.load(fname_pcorr, allow_pickle=True)
                id = d['id']
                zscore = d['z_score_pearson_corr']

                # find which values are above threshold
                idx = np.where(zscore>pcorr_thresh)[0]
                cm[k,idx] = zscore[idx]

                # this is a symmetric matrix
                cm[idx,k] = zscore[idx]


            # make a network graph from the cm matrix
            G = nx.Graph(cm)

            # remove self connections
            G.remove_edges_from(nx.selfloop_edges(G))

            # remove isolates
            G.remove_nodes_from(list(nx.isolates(G)))

            # find largest component in G
            G_largest = G.subgraph(max(nx.connected_components(G), key=len))

            # compute the diameter of G
            diam = nx.diameter(G_largest)

            # grab the clustering coefficient of G for all nodes
            clust = nx.clustering(G_largest)
            # compute the mean clustering coefficient from clust dict
            if use_median:
                clust = np.median(list(clust.values()))
            else:
                clust = np.mean(list(clust.values()))

            # compute the average shortest path length
            path_len = nx.average_shortest_path_length(G_largest)
                
            # compute the average degree
            deg = nx.average_degree_connectivity(G_largest)
            if use_median:
                deg = np.median(list(deg.values()))
            else:
                deg = np.mean(list(deg.values()))

            # compute the pagerank
            pr = nx.pagerank(G_largest)
            if use_median:
                pr = np.median(list(pr.values()))
            else:
                pr = np.mean(list(pr.values()))

            # Save the graph to a Pickle file
            fname_pkl = os.path.join(dir_main, 'graph.pkl')
            with open(fname_pkl, "wb") as f:
                pickle.dump(G, f)
        
            np.savez(fname_out,
                    diameters = diam,
                    clustering = clust,
                    aspl = path_len,
                    degree = deg,
                    pagerank = pr,
                    pcorr_thresh = pcorr_thresh)


        else:
            d = np.load(fname_out, allow_pickle=True)
            diam = d['diameters']
            clust = d['clustering']
            path_len = d['aspl']
            deg = d['degree']
            pr = d['pagerank']

            #
            G = pickle.load(open(os.path.join(os.path.split(fname_out)[0], 
                                              'graph.pkl')), "rb")

        # add to lists
        diameters.append(diam)
        clustering.append(clust)
        aspl.append(path_len)
        degree.append(deg)
        pagerank.append(pr)
    
        #
        ctr+=1

    # save master list
    dir_out = os.path.join(root_dir, 
                           animal_id, 
                           'correlations')
    if os.path.exists(dir_out)==False:
        os.mkdir(dir_out)

    # save master list
    # load yaml file and get session_ids field
    fname_out_animal = os.path.join(root_dir, 
                                    animal_id, 
                                    'correlations',
                                    'graph_metrics.npz')
    
    np.savez(fname_out_animal,
            pcorr_thresh = pcorr_thresh,
            diameters = diameters,
            clustering = clustering,
            aspl = aspl,
            degree = degree,
            pagerank = pagerank
            )


#
recompute_graphs = False
pcorr_thresh = 0.99
use_median = False

compute_graph_metrics(animal_id,
                        recompute_graphs,
                        pcorr_thresh,
                        use_median)


print ("DONE")



TypeError: load() takes exactly 1 positional argument (2 given)

In [11]:
#
def proc_data(ax, 
              data,
              data_name,
              clr,
              text=None):

    ax.set_title(data_name)
    ax.plot(data,
             linewidth=2,
             color=clr)
    # fit linear regression to the data
    x = np.arange(len(data))
    y = np.array(data)
    idx = np.isfinite(y)
    z = np.polyfit(x[idx], y[idx], 1)
    p = np.poly1d(z)

    rr,pp = scipy.stats.pearsonr(x,y)
    #
    ax.plot(x,p(x),
             '--',
             linewidth=2,
             c=clr,
             label = ('r='+str(np.round(rr,2))+' p='+str(np.round(pp,2))) if text is None else text
             )
    

#
clrs = ['blue','navy','lightblue','red','orange']

# plot the distributions
plt.figure(figsize=(10,10))
plt.suptitle(c.animal_id)



#
for ctr, animal_id in enumerate(animal_ids):
    fname = os.path.join(c.root_dir, 
                        animal_id,
                        'correlations',
                        'graph_metrics.npz')

    #
    data = np.load(fname, allow_pickle=True)
    diameters = data['diameters']
    clustering = data['clustering']
    aspl = data['aspl']
    degree = data['degree']
    pagerank = data['pagerank']

    # 
    data_array = [diameters, clustering, aspl, degree, pagerank]
    data_names = ['Diameter', 'Clustering', 'Ave path len', 'Degree', 'Pagerank']

    #
    for k in range(5):
        ax = plt.subplot(2,3,k+1)

        proc_data(ax,
                    data_array[k],
                    data_names[k],
                    clrs[ctr],
                    text = animal_id if k==0 else None
                    )
        ax.legend()

        # label xaxis with session starting with 'day0' and followed by 'bmiN' where n
        ax.set_xlabel('Session #')

plt.show()

/home/cat/.conda/envs/bmitools/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [17]:
# plot the single cell correlations over time
#

animal_id = animal_ids[0]

# load the yaml file to get session_ids
fname_yaml = os.path.join(root_dir, 
                          animal_id, 
                          str(animal_id)+'.yaml')

# load yaml file
d = yaml.load(open(fname_yaml), 
              Loader=yaml.FullLoader)
session_ids = d['session_ids']

# make viridis map over len(session_ids)
cmap = matplotlib.cm.get_cmap('viridis')
colors = [cmap(i) for i in np.linspace(0, 1, len(session_ids))]

# find # of cells in recording
fname = os.path.join(root_dir,
                    animal_id,
                    str(session_ids[0]),
                    'plane0',
                    'merged',
                    'F.npy')
n_cells = np.load(fname).shape[0]
print ("# cells: ", n_cells)


#
cell_modes = []
for cell_id in range(n_cells):

    #
    temp_mode = []
    for ctr, session_id in enumerate(session_ids):
        fname = os.path.join(root_dir,
                            animal_id,
                            str(session_id),
                            'plane0',
                           # 'merged',
                            'correlations',
                            str(cell_id)+'.npz')
        
        #
        try:
            dd = np.load(fname, allow_pickle=True)
        except:
            continue

        pcorr = dd['pearson_corr']

        # remove nans
        idx = np.isnan(pcorr)==False
        pcorr = pcorr[idx]
        # # get mode of pcor

        # find 90th percentile of the pcorr distrib
        try:
            percentile = np.percentile(pcorr, 90)
        except:
            print ("Error", pcorr)

        temp_mode.append(percentile)  

        #break
    #
    cell_modes.append(temp_mode)

    #break


/tmp/ipykernel_2156899/189079872.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = matplotlib.cm.get_cmap('viridis')


FileNotFoundError: [Errno 2] No such file or directory: '/media/cat/8TB/donato/bmi/DON-019683/20240119/plane0/merged/F.npy'

In [18]:
#
print (len(cell_modes))
print (len(cell_modes[0]))


# plot violion plots using seaborn
import seaborn as sns
import pandas as pd

#
n_sess = 9
sns.set(style="whitegrid")
plt.figure(figsize=(8, 4))
sns.violinplot(data=cell_modes[:n_sess], 
               #inner="quart", 
               #palette="Set3"
               )

#
sess = ['day0', 'bmi1', 'bmi2', 'bmi3', 'bmi4', 'bmi5', 'bmi6', 'bmi7', 'bmi8']
plt.xticks(np.arange(n_sess), sess[:n_sess], rotation=0)

plt.ylabel('Median pairwise correlation')

plt.suptitle(animal_id)

#
plt.show()




0


IndexError: list index out of range

In [19]:
# plot the single cell correlations over time

animal_id = 'DON-015801'

# load the yaml file to get session_ids
fname_yaml = os.path.join(root_dir, 
                          animal_id, 
                          str(animal_id)+'.yaml')

# load yaml file
d = yaml.load(open(fname_yaml), 
              Loader=yaml.FullLoader)
session_ids = d['session_ids']

# make viridis map over len(session_ids)
cmap = matplotlib.cm.get_cmap('viridis')
colors = [cmap(i) for i in np.linspace(0, 1, len(session_ids))]

# find # of cells in recording
fname = os.path.join(root_dir,
                    animal_id,
                    str(session_ids[0]),
                    'plane0',
                    'merged',
                    'F.npy')
n_cells = np.load(fname).shape[0]
print ("# cells: ", n_cells)

if c.use_non_merged==True:
    print (" THIS IS A NON-MERGED DATASET")
    print (" ... can't do multi-session tracking... ")
else:


    #
    n_plots = 100
    plt.figure(figsize=(15,15))
    ctra =0
    #cell_ids = np.arange()
    cell_modes = []
    for cell_id in range(n_cells):
        ax = plt.subplot(int(np.sqrt(n_plots)),
                        int(np.sqrt(n_plots)),
                        ctra+1)

        #
        temp_mode = []
        for ctr, session_id in enumerate(session_ids):
            fname = os.path.join(root_dir,
                                animal_id,
                                str(session_id),
                                'plane0',
                                'merged',
                                'correlations',
                                str(cell_id)+'.npz')
            
            #
            try:
                dd = np.load(fname, allow_pickle=True)
            except:
                continue

            pcorr = dd['pearson_corr']
            #print (pcorr)

            # make a histogram
            # y = np.histogram(pcorr, bins=np.arange(-0.1,0.25,0.01))   M1 mice
            y = np.histogram(pcorr, bins=np.arange(-0.03,0.2,0.003))
            xx = y[1][:-1]
            yy = y[0]

            # plot the histogram
            plt.plot(xx,yy, 
                    color=colors[ctr],
                    alpha=0.75
                    #linewidth=2
                    )
            plt.yticks([])

            plt.title(str(cell_id), fontsize=6)

            # 
            if ctra<90:
                plt.xticks([])
            else:
                # make tick size smaller
                plt.xticks(fontsize=6)
        #
        ctra+=1

        if ctra==n_plots:
            plt.xlabel('Pearson correlation')
            plt.ylabel('# pairs')
            fname = os.path.join(root_dir,
                                animal_id,
                                'correlations',
                                str(cell_id-n_plots+1)+'_correlations.png')
            plt.savefig(fname,dpi=300)
            plt.close()

            #
            plt.figure(figsize=(15,15))
            ctra=0


    #
    plt.xlabel('Pearson correlation')
    plt.ylabel('# pairs')
    fname = os.path.join(root_dir,
                        animal_id,
                        'correlations',
                        str(cell_id-n_plots+1)+'_correlations.png')


    plt.savefig(fname,dpi=300)
    plt.close()







# cells:  712
 THIS IS A NON-MERGED DATASET
 ... can't do multi-session tracking... 


/tmp/ipykernel_2156899/2042534127.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = matplotlib.cm.get_cmap('viridis')


In [118]:
#
# Load data from specific animal / day
animal_id = 'DON-015801'
session_id = '20230615'

dir_main = os.path.join(root_dir,
                        animal_id,
                        session_id,
                        'plane0',
                        'correlations')

# find all.npz files in dir_main
files = os.listdir(dir_main)
files = [file for file in files if 'graph' not in file]
print ("# files: ", len(files))

# make a connectivity matrix
cm = np.zeros((len(files), len(files)), 'float32')

#
pcorr_thresh = 5
for k in range(len(files)):
        
    # load cell correlations
    fname_pcorr = os.path.join(dir_main, files[k])
    d = np.load(fname_pcorr, allow_pickle=True)
    id = d['id']
    zscore = d['z_score_pearson_corr']
    
    # find which values are above threshold
    idx = np.where(zscore>pcorr_thresh)[0]
    cm[k,idx] = zscore[idx]
    
    # this is a symmetric matrix
    cm[idx,k] = zscore[idx]

    # make a boolean matrix
    cm_bool = np.zeros((cm.shape),'bool')
    cm_bool[cm>0]=1


#
plt.figure()
plt.imshow(cm_bool,
           vmin=0,
           vmax=1,
           cmap='binary')
plt.ylim(0,cm_bool.shape[0])
plt.colorbar()
plt.show()


# files:  472


In [2]:
import numpy as np
d = np.load('/media/cat/8TB/donato/bmi/DON-014451/20230330/plane0/F.npy')
print (d.shape)

(699, 90000)


In [22]:
########################################################
########### LOAD SPREADSHEETS AND METADATA #############
########################################################

# adds trial information; eventually should also add bursts etc...
c.fix_spreadsheet()

# 


processing session:  20230130
 ... adding BLANK tone_state
 ... adding BLANK ensemble_state
 ... adding COPY OF post_reward_state as reward_state


 60%|██████    | 54123/89951 [00:00<00:00, 270877.34it/s]

... found rewarded trial at frame:  901  time:  30.033333333333335 s


100%|██████████| 89951/89951 [00:00<00:00, 269949.99it/s]

# of trials:  1 # of rewarded trials:  1
reward_times and trials[idx][:,1] are not identical
reward_times:  [ 8787 33064]
trials[idx][:,1]:  [901]
Exiting...


In [19]:
print (c.reward_state )

plt.figure()
plt.plot(c.reward_state)
plt.plot(c.post_reward_state)
plt.show()

[0 0 0 ... 0 0 0]


In [ ]:

# 
print (c.session_ids)
c.fix_spreadsheet_missing_vals = True

#
print (c.session_types)
c.load_spreadsheet_multi()

# also save the res array in the same folder
c.load_best_matches()


In [117]:
########################################################
########## LOAD PAIRWISE CORRELATION DATA ##############
########################################################
# this is precomputed on the cluster 
#
def zero_ca_artifacts(fname):
    d = np.load(fname, 
                allow_pickle=True)

    # load the upphase_bin
    F_upphase = d['F_upphase']
    F_detrended = d['F_detrended']
    F_filtered = d['F_filtered']
 

    # make mua
    mua = np.sum(F_upphase,0)

    thresh = 60
    time_thresh = 30*2

    # find periods where mua > thresh and that are longer than 10 frames
    idx = np.where(mua>thresh)[0]

    # find contiguous periods within idx and measure duration
    bouts = []
    start = idx[0]
    #
    for k in range(1,len(idx)-1,1):
        if idx[k]==idx[k+1]-1:
            continue
        else:
            end = idx[k]
            if (end-start)>time_thresh:
                bouts.append([start, end])
            start = idx[k+1]

    #
    bouts = np.array(bouts)
    print ("# bouts to be zeroed out: ", bouts.shape[0])    

    #
    for bout in bouts:
        F_upphase[:,bout[0]:bout[1]]=0
        F_detrended[:,bout[0]:bout[1]]=0
        F_filtered[:,bout[0]:bout[1]]=0

    # save thsi files separately for each session
    fname_out = os.path.join(os.path.split(fname)[0],
                             'F_upphase.npy')
    np.save(fname_out, F_upphase)

    fname_out = os.path.join(os.path.split(fname)[0],
                                'F_detrended.npy')
    np.save(fname_out, F_detrended)

    fname_out = os.path.join(os.path.split(fname)[0],
                                'F_filtered.npy')
    np.save(fname_out, F_filtered)

    

#
fname = '/media/cat/8TB/donato/bmi/DON-015801/20230614/plane0/binarized_traces.npz'
fname = '/media/cat/8TB/donato/bmi/DON-015801/20230614/plane0/merged/binarized_traces.npz'
zero_ca_artifacts(fname)

#


# bouts to be zeroed out:  3


In [125]:
##############################################################
######## COMPUTE REWARD TRIGGERED RESPONSE OF ROIs ###########
##############################################################
# 
from utils_calcium import plot_psth_rois_only

#
def init_fig():
    plt.figure(figsize=(20,12))
    axes = []
    for k in range(4):
        ax=plt.subplot(2,2,k+1)
        axes.append(ax) 

    return axes

# 
window = 300
start_cell=0
show_random = True
idx_cells = None
smoothing = True
plot_separately = False
c.plot_rois_only = True
c.cells_to_plot = c.best_matches
c.cells_to_plot = np.random.choice(np.arange(c.sessions[0].F_filtered.shape[0]),
                                   4,
                                   replace=False)
#
print ("c.cells_to_plot: ", c.cells_to_plot)
#
axes = init_fig()

#
for session_id in np.arange(1,9,1):

    #
    psths_avg, psths_shuffled_avg, idx_cells, n_bursts  = get_reward_triggered_psth(session_id, 
                                                                                    c,
                                                                                    window,
                                                                                    idx_cells)
    #
    plot_psth_rois_only(c,
                        psths_avg,
                        psths_shuffled_avg,
                        session_id,
                        axes,
                        start_cell,
                        idx_cells,
                        show_random,
                        smoothing,
                        window
                        )


    # reset axes
    if plot_separately:
        #
        plt.suptitle(c.animal_id+" session: "+str(session_id),fontsize=20)
        #
        plt.savefig('/home/cat/'+str(start_cell)+"_session_"+str(session_id)+'.png',dpi=300)
        plt.close()

        axes = init_fig()

#
plt.suptitle(c.animal_id+" session: "+str(session_id),fontsize=20)
#
plt.savefig('/home/cat/'+str(start_cell)+"_session_"+str(session_id)+'.png',dpi=300)
plt.close()


c.cells_to_plot:  [270 321 223 196]
session:  1  of 8 ,  reward times:  (33,)
session:  2  of 8 ,  reward times:  (41,)
session:  3  of 8 ,  reward times:  (33,)
session:  4  of 8 ,  reward times:  (45,)
session:  5  of 8 ,  reward times:  (64,)
session:  6  of 8 ,  reward times:  (39,)
session:  7  of 8 ,  reward times:  (56,)
session:  8  of 8 ,  reward times:  (56,)


In [126]:
##############################################################
######## COMPUTE REWARD TRIGGERED RESPONSE OF CELLS ##########
##############################################################
window = 30*6
psth_array = []
idx_cells = None
global_order = False
session_ids = np.arange(1,len(c.session_ids),1)

#
n_bursts_array = []
for session_id in session_ids:
#for session_id in [1,7]:

    #
    psths_avg, psths_shuffled_avg, idx_cells, n_bursts  = get_reward_triggered_psth(session_id, 
                                                                                    c,
                                                                                    window,
                                                                                    idx_cells)
    
    #
    n_bursts_array.append(n_bursts)
    
    #
    psth_array.append(psths_avg[idx_cells[::-1]])

    if global_order==False:
        idx_cells = None

#
psth_array = np.array(psth_array)
print (psth_array.shape)

#
vmax = 0.3
sua = plot_multi_session_psth_imshow(c,
                                session_ids,
                                psth_array,
                                window,
                                n_bursts_array,
                                vmax=vmax)


session:  1  of 8 ,  reward times:  (33,)
session:  2  of 8 ,  reward times:  (41,)
session:  3  of 8 ,  reward times:  (33,)
session:  4  of 8 ,  reward times:  (45,)
session:  5  of 8 ,  reward times:  (64,)
session:  6  of 8 ,  reward times:  (39,)
session:  7  of 8 ,  reward times:  (56,)
session:  8  of 8 ,  reward times:  (56,)
(8, 540, 360)


In [5]:
#######################################
#######################################
#######################################

# binarize the ensembel state    
c.plotting = False
c.scale_threshold = 0.05 #                 # we want to decrease the burst detection threshold manually - because the 
                                            # automatic threshold detection is too conservative 
c.dff_threshold = 0.1
c.percentile_threshold = 0.999

#
#print ("TODO: need to add the standaline binairzation code to the binarize2pcalcium package....")
#print (" This is generally a good idea anyways for doing single time sieres extraction")

# right now only binarizing upphase - if want to do negative phase have to invert the signal
c.sample_rate = 30
c.binarize_ensemble_state_multi()


fitting mode to physics:   0%|          | 0/1 [00:00<?, ?it/s]

/home/cat/.conda/envs/bmi/lib/python3.8/site-packages/binarize2pcalcium/binarize2pcalcium.py:4026: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_mode = scipy.stats.mode(F_filtered2)[0]
fitting mode to physics: 100%|██████████| 1/1 [00:00<00:00, 81.99it/s]


cumsum:  [4.63649033e-15 9.27561305e-15 1.39173696e-14 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]


binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1/1 [00:00<00:00, 79.64it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1/1 [00:00<00:00, 74.52it/s]
fitting mode to physics: 100%|██████████| 1/1 [00:00<00:00, 76.03it/s]


cumsum:  [5.01316877e-10 1.00276836e-09 1.50435447e-09 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]


...

binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1/1 [00:00<00:00, 61.93it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1/1 [00:00<00:00, 53.58it/s]
fitting mode to physics: 100%|██████████| 1/1 [00:00<00:00, 76.58it/s]


cumsum:  [1.26496650e-17 2.53084175e-17 3.79762638e-17 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]


binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1/1 [00:00<00:00, 63.18it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1/1 [00:00<00:00, 56.17it/s]
fitting mode to physics: 100%|██████████| 1/1 [00:00<00:00, 66.88it/s]


cumsum:  [4.13738239e-19 8.27809700e-19 1.24221465e-18 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]


binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1/1 [00:00<00:00, 53.34it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1/1 [00:00<00:00, 40.86it/s]


In [9]:
dd = np.load('/media/cat/8TB/donato/bmi/DON-011733/20230307/ensembles/ensembles_matches_bmi_to_suite2p.npz', allow_pickle=True)

#
plt.figure()
ax=plt.subplot(411)
#
e1_0 = dd ['ensemble1_0_bmi']
f0 = np.median(e1_0,0)
e1_0 = (e1_0-f0)/f0
e1_0_2p = dd['ensemble1_0_suite2p']
plt.plot(e1_0, 'r')
plt.plot(e1_0_2p, 'b')

#
ax=plt.subplot(412)
e1_1 = dd ['ensemble1_1_bmi']
f1 = np.median(e1_1,0)
e1_1 = (e1_1-f1)/f1
e1_1_2p = dd['ensemble1_1_suite2p']
plt.plot(e1_1, 'r')
plt.plot(e1_1_2p, 'b')

#
ax=plt.subplot(413)
e2_0 = dd ['ensemble2_0_bmi']
f2 = np.median(e2_0,0)
e2_0 = (e2_0-f2)/f2
e2_0_2p = dd['ensemble2_0_suite2p']
plt.plot(e2_0, 'r')
plt.plot(e2_0_2p, 'b')

#
ax=plt.subplot(414)
e2_1 = dd ['ensemble2_1_bmi']
f3 = np.median(e2_1,0)
e2_1 = (e2_1-f3)/f3
e2_1_2p = dd['ensemble2_1_suite2p']
plt.plot(e2_1, 'r')
plt.plot(e2_1_2p, 'b')


plt.show()
